In [16]:
import pandas as pd
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_community.vectorstores import FAISS 
from langchain_community.llms import HuggingFacePipeline  
from langchain.chains import RetrievalQA
from langchain.embeddings import OpenAIEmbeddings
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain.prompts import PromptTemplate

import os

In [ ]:
loader = TextLoader('../data/cardapio_preparado.txt', encoding='utf-8')
docs = loader.load()

splitter = TokenTextSplitter(
    chunk_size=300, chunk_overlap=100
)

documents = splitter.split_documents(docs)

In [9]:
embedding_model = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents, embedding_model)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [28]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(
    model_name="gpt-4o-mini",  # ou gpt-4, gpt-4o-mini, gpt-3.5-turbo
    temperature=0,
    max_tokens=256,
)

In [30]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.schema import Document

# Define o modelo da OpenAI
llm = ChatOpenAI(
    model_name="gpt-4o",  # ou gpt-4, gpt-4o-mini, gpt-3.5-turbo
    temperature=0,
    max_tokens=512,
)

# Template de prompt
template = """
Você é um assistente especializado em cardápios de restaurantes.

Baseado nas informações abaixo, responda de forma clara, direta e objetiva a pergunta do usuário.
Se a informação não estiver presente, informe que não há dados disponíveis.
Se a pergunta não for sobre cardápios, informe que você não pode ajudar com isso.

Contexto:
{context}

Pergunta:
{question}

Resposta:
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

# Criação da cadeia de QA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)



In [31]:

# Exemplo de uso:
resposta = qa_chain.invoke({"query": "Tem suco ai?"})


In [32]:
print(resposta['result'])

Sim, há vários sucos disponíveis no cardápio. Aqui estão alguns exemplos:

- Laranja C/ Mamão, código 522, a partir de 12,90.
- Limão C/ Água, código 519, a partir de 10,90.
- Limonada Suíça, código 525, a partir de 12,90.
- Mamão C/ Água, código 510, a partir de 9,90.
- Manga C/ Água, código 511, a partir de 10,90.
- Maracujá C/ Água, código 512, a partir de 10,90.
- Abacaxi C/ Água, código 501, a partir de 10,90.

E muitos outros.


In [27]:
resposta['source_documents']

[Document(id='cfbb9048-c47a-4128-9c9b-6a6d4b31fd5a', metadata={'source': '../data/cardapio_preparado.txt'}, page_content=', tem o código: 522, é da categoria SUCO, com valor a partir de 12,90. O produto: Laranja C/ Morango, tem o código: 524, é da categoria SUCO, com valor a partir de 12,90. O produto: Limao C/ Agua, tem o código: 519, é da categoria SUCO, com valor a partir de 10,90. O produto: Limonada Suiça, tem o código: 525, é da categoria SUCO, com valor a partir de 12,90. O produto: Limoneto, tem o código: 149, é da categoria REFRIGERANTE, com valor a partir de 7,90. O produto: Limão C/ Leite, tem o código: 547, é da categoria SUCO, com valor a partir de 11,90. O produto: Linguiça, tem o código: 109, é da categoria CARNES, com valor a partir de 145,00. O produto: Loirinha Cookies, tem o código: 722, é da categoria SORVETE, com valor a partir de 10,50.'),
 Document(id='bfcf7d33-c16a-4995-9337-eab2d433934e', metadata={'source': '../data/cardapio_preparado.txt'}, page_content='VEJA